# Data Science Notes

By Rey G.

Using the Protein Structure Data (https://www.kaggle.com/datasets/shahir/protein-data-set), these notes are aimed to learn some basic aspects of data science.

In [1]:
import sqlalchemy as sq
import pandas as pd

In [2]:
engine = sq.create_engine('sqlite:///database.db')

Load the CSV to DataFrame, then load them as db for sqlite

In [3]:
seq_df = pd.read_csv("pdb_data_seq.csv")
dups_df = pd.read_csv("pdb_data_no_dups.csv")

In [4]:
seq_df.to_sql(name='seq', con=engine,if_exists='replace')
dups_df.to_sql(name='dups', con=engine,if_exists='replace')

141401

In [5]:
def query_data(query : str, engine):
    q = engine.execute(query)
    df = pd.DataFrame(q.fetchall())
    df.columns = q.keys()
    return df

## Section 1: Data Cleansing

The heart of all machine learning algorithm lies in data. Data that are too jumbled would be too much of a hassle to work with. In this section, we will go to a gentle walkthrough of how to use SQL and how we can use them to clean our data.

In my experience, Pandas are relatively easier to use than SQL, so this notes assume the user's basic knowledge of Pandas command instead of SQL.

Not covered (yet) in this notes are data types, Create Table/Procedure, and some more. Only basic SQL operations such as SELECT, where clause, table joining, subqueries, and aggregations will be covered here.

### a. Where it all starts
In order to query from a database, one of the simplest (that I know of) to fetch data into a query is written like these:

In [6]:
query = '''
SELECT * FROM seq
'''
seq_data = query_data(query, engine)
all_data = seq_data.drop('index', axis=1)
all_data

,structureId,chainId,sequence,residueCount,macromoleculeType
0,100D,A,CCGGCGCCGG,20,DNA/RNA Hybrid
1,100D,B,CCGGCGCCGG,20,DNA/RNA Hybrid
2,101D,A,CGCGAATTCGCG,24,DNA
3,101D,B,CGCGAATTCGCG,24,DNA
4,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,Protein
...,...,...,...,...,...
467299,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388,Protein
467300,9XIM,A,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572,Protein
467301,9XIM,B,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572,Protein
467302,9XIM,C,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572,Protein


SELECT * statement basically calls all the columns inside the table, which can be very expensive if you query them using big platforms such as Bigquery. Try to only pick columns you need, for example:

In [7]:
query = '''
SELECT structureId, sequence FROM seq
'''
seq_data = query_data(query, engine)
seq_data

,structureId,sequence
0,100D,CCGGCGCCGG
1,100D,CCGGCGCCGG
2,101D,CGCGAATTCGCG
3,101D,CGCGAATTCGCG
4,101M,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
...,...,...
467299,9XIA,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...
467300,9XIM,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...
467301,9XIM,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...
467302,9XIM,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...


Neat, right? You can also select only the unique values of the data, using DISTINCT.

In [8]:
query = '''
SELECT distinct structureId, sequence FROM seq
'''
seq_data = query_data(query, engine)
seq_data

,structureId,sequence
0,100D,CCGGCGCCGG
1,101D,CGCGAATTCGCG
2,101M,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
3,102D,CGCAAATTTGCG
4,102L,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...
...,...,...
251715,9RSA,KETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTF...
251716,9RUB,MDQSSRYVNLALKEEDLIAGGEHVLCAYIMKPKAGYGYVATAAHFA...
251717,9WGA,ERCGEQGSNMECPNNLCCSQYGYCGMGGDYCGKGCQNGACWTSKRC...
251718,9XIA,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...


Or you can do it like:

In [9]:
query = '''
SELECT distinct macromoleculeType FROM seq
'''
seq_data = query_data(query, engine)
seq_data

,macromoleculeType
0,DNA/RNA Hybrid
1,DNA
2,Protein
3,Protein#DNA
4,DNA#RNA
5,RNA
6,DNA#DNA/RNA Hybrid
7,None
8,Protein#RNA
9,RNA#DNA/RNA Hybrid


Now, let's load another data.

In [10]:
query = '''
SELECT * FROM dups
'''
seq_data = query_data(query, engine)
seq_data

,index,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear
0,0,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,1.78,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0
1,1,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,None,NaN,2.00,38.45,None,NaN,1995.0
2,2,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,None,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
3,3,102D,DNA,X-RAY DIFFRACTION,DNA,24,2.20,7637.17,"VAPOR DIFFUSION, SITTING DROP",277.0,2.28,46.06,"pH 7.00, VAPOR DIFFUSION, SITTING DROP, temper...",7.0,1995.0
4,4,102L,HYDROLASE(O-GLYCOSYL),X-RAY DIFFRACTION,Protein,165,1.74,18926.61,None,NaN,2.75,55.28,None,NaN,1993.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141396,141396,9RUB,LYASE(CARBON-CARBON),X-RAY DIFFRACTION,Protein,932,2.60,101838.68,None,NaN,2.38,48.29,None,NaN,1991.0
141397,141397,9TNA,T-RNA,X-RAY DIFFRACTION,None,0,NaN,24244.34,None,NaN,3.17,61.18,None,NaN,1986.0
141398,141398,9WGA,LECTIN (AGGLUTININ),X-RAY DIFFRACTION,Protein,342,1.80,34270.22,None,NaN,2.50,50.76,None,NaN,1990.0
141399,141399,9XIA,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,388,1.90,43542.29,None,NaN,2.79,55.93,None,NaN,1989.0


Now *that's* a shitload of data! Sometimes you would like to only select certain type of data from your dataset. To do this, use the WHERE clause.

In [11]:
query = '''
SELECT * FROM dups
where macromoleculeType = 'Protein'
'''
seq_data = query_data(query, engine)
seq_data

,index,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear
0,2,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,None,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
1,4,102L,HYDROLASE(O-GLYCOSYL),X-RAY DIFFRACTION,Protein,165,1.74,18926.61,None,NaN,2.75,55.28,None,NaN,1993.0
2,5,102M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,1.84,18010.64,None,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
3,7,103L,HYDROLASE(O-GLYCOSYL),X-RAY DIFFRACTION,Protein,167,1.90,19092.72,None,NaN,2.70,54.46,None,NaN,1993.0
4,8,103M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18093.78,None,NaN,3.09,60.30,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127793,141395,9RSA,HYDROLASE (PHOSPHORIC DIESTER),X-RAY DIFFRACTION,Protein,248,1.80,27987.16,None,NaN,2.25,45.45,None,NaN,1990.0
127794,141396,9RUB,LYASE(CARBON-CARBON),X-RAY DIFFRACTION,Protein,932,2.60,101838.68,None,NaN,2.38,48.29,None,NaN,1991.0
127795,141398,9WGA,LECTIN (AGGLUTININ),X-RAY DIFFRACTION,Protein,342,1.80,34270.22,None,NaN,2.50,50.76,None,NaN,1990.0
127796,141399,9XIA,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,388,1.90,43542.29,None,NaN,2.79,55.93,None,NaN,1989.0


You have filtered only the protein part of the macromolecule, and there's around 120k-ish data of that type. You can put some conditionals to further put grains in your clause:

In [12]:
query = '''
SELECT distinct * FROM dups
where macromoleculeType = 'Protein'
and classification like '%OXYGEN%'
and publicationYear <= 1998
'''
seq_data = query_data(query, engine)
seq_data

,index,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear
0,226,1A00,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,574,2.00,64565.11,None,NaN,2.32,47.00,10.5% PEG 6000 10 MM POTASSIUM PHOSPHATE PH 7....,7.0,1998.0
1,227,1A01,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,574,1.80,64380.92,None,NaN,2.19,44.00,2.3 M AMMONIUM SULFATE 0.3 M AMMONIUM PHOSPHAT...,6.5,1998.0
2,256,1A0U,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,574,2.14,64611.18,None,NaN,2.46,50.00,10.5% PEG 6000 10 MM POTASSIUM PHOSPHATE PH 7....,7.0,1998.0
3,261,1A0Z,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,574,2.00,64611.18,None,NaN,2.32,47.00,10.5% PEG 6000 10 MM POTASSIUM PHOSPHATE PH 7....,7.0,1998.0
4,382,1A4F,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,287,2.00,32972.52,None,NaN,2.71,54.00,None,NaN,1996.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,141180,7CPP,OXIDOREDUCTASE(OXYGENASE),X-RAY DIFFRACTION,Protein,414,2.00,47315.52,None,NaN,2.20,44.16,None,NaN,1989.0
321,141182,7ENL,CARBON-OXYGEN LYASE,X-RAY DIFFRACTION,Protein,436,2.20,46901.05,None,NaN,2.67,53.91,None,NaN,1991.0
322,141193,7HBI,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,292,1.60,33219.66,None,NaN,2.27,45.00,PROTEIN WAS CRYSTALLIZED FROM 2.3M NA/K PHOSPH...,7.5,1998.0
323,141259,8ACN,LYASE(CARBON-OXYGEN),X-RAY DIFFRACTION,Protein,754,2.00,83292.66,None,NaN,2.88,57.24,None,NaN,1992.0


More about the LIKE operator can be read here: https://www.w3schools.com/sql/sql_like.asp. This is useful whenever you would like to filter out strings data but you don't really know the exact, full words of it.

### EXERCISE 1

Protein structure research, just like any other types of research projects, are often tied with their year of publication. Make a query that filter the dups dataset before the year 1999. Without having to invoke statistics yet, what kind of patterns do you see in the data?

Answer below:

In [13]:
#Answer goes here. Explanations should be given in markdowns.

### b. Subqueries, CTEs, and whatnot

Subqueries are basically a query inside a query. Let me show you:

In [14]:
query = '''
SELECT * from (
select distinct * FROM dups
where macromoleculeType = 'Protein'
and classification like '%OXYGEN%'
)
'''
seq_data = query_data(query, engine)
seq_data

,index,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear
0,2,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,None,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
1,5,102M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,1.84,18010.64,None,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
2,8,103M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18093.78,None,NaN,3.09,60.30,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
3,11,104M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,153,1.71,18030.63,None,NaN,1.87,34.30,"3.0 M AMMONIUM SULFATE, UNBUFFERED, pH 7.0",7.0,1999.0
4,13,105M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,153,2.02,18030.63,None,NaN,1.83,33.00,"3.0 M AMMONIUM SULFATE, UNBUFFERED, pH 9.0",9.0,1999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,141180,7CPP,OXIDOREDUCTASE(OXYGENASE),X-RAY DIFFRACTION,Protein,414,2.00,47315.52,None,NaN,2.20,44.16,None,NaN,1989.0
1064,141182,7ENL,CARBON-OXYGEN LYASE,X-RAY DIFFRACTION,Protein,436,2.20,46901.05,None,NaN,2.67,53.91,None,NaN,1991.0
1065,141193,7HBI,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,292,1.60,33219.66,None,NaN,2.27,45.00,PROTEIN WAS CRYSTALLIZED FROM 2.3M NA/K PHOSPH...,7.5,1998.0
1066,141259,8ACN,LYASE(CARBON-OXYGEN),X-RAY DIFFRACTION,Protein,754,2.00,83292.66,None,NaN,2.88,57.24,None,NaN,1992.0


This achieves a degree of freedom where you can query as many tables as possible with a short script. What about CTE, then?

In [15]:
query = '''
with distin as (
select distinct * FROM dups
where macromoleculeType = 'Protein'
and classification like '%OXYGEN%'
and publicationYear <= 1998
)
SELECT * from distin
'''
seq_data = query_data(query, engine)
seq_data

,index,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear
0,226,1A00,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,574,2.00,64565.11,None,NaN,2.32,47.00,10.5% PEG 6000 10 MM POTASSIUM PHOSPHATE PH 7....,7.0,1998.0
1,227,1A01,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,574,1.80,64380.92,None,NaN,2.19,44.00,2.3 M AMMONIUM SULFATE 0.3 M AMMONIUM PHOSPHAT...,6.5,1998.0
2,256,1A0U,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,574,2.14,64611.18,None,NaN,2.46,50.00,10.5% PEG 6000 10 MM POTASSIUM PHOSPHATE PH 7....,7.0,1998.0
3,261,1A0Z,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,574,2.00,64611.18,None,NaN,2.32,47.00,10.5% PEG 6000 10 MM POTASSIUM PHOSPHATE PH 7....,7.0,1998.0
4,382,1A4F,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,287,2.00,32972.52,None,NaN,2.71,54.00,None,NaN,1996.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,141180,7CPP,OXIDOREDUCTASE(OXYGENASE),X-RAY DIFFRACTION,Protein,414,2.00,47315.52,None,NaN,2.20,44.16,None,NaN,1989.0
321,141182,7ENL,CARBON-OXYGEN LYASE,X-RAY DIFFRACTION,Protein,436,2.20,46901.05,None,NaN,2.67,53.91,None,NaN,1991.0
322,141193,7HBI,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,292,1.60,33219.66,None,NaN,2.27,45.00,PROTEIN WAS CRYSTALLIZED FROM 2.3M NA/K PHOSPH...,7.5,1998.0
323,141259,8ACN,LYASE(CARBON-OXYGEN),X-RAY DIFFRACTION,Protein,754,2.00,83292.66,None,NaN,2.88,57.24,None,NaN,1992.0


Same, right? And nobody can forbid you from combining both CTE and 

### c. Table Joining

Joining tables in SQL is an act of trying to match the tables on common columns. There are many types of joins, but for the sake of simplicty I only show two of them (cuz SQLite kinda sucks), demonstrated below:

i. inner join: Match only the columns

In [16]:
query = '''
SELECT * FROM dups d
inner join seq s
on d.structureId = s.structureId
'''
seq_data = query_data(query, engine)
seq_data

,index,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,...,densityPercentSol,pdbxDetails,phValue,publicationYear,index,structureId,chainId,sequence,residueCount,macromoleculeType
0,0,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,...,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0,0,100D,A,CCGGCGCCGG,20,DNA/RNA Hybrid
1,0,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,...,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0,1,100D,B,CCGGCGCCGG,20,DNA/RNA Hybrid
2,1,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,None,NaN,...,38.45,None,NaN,1995.0,2,101D,A,CGCGAATTCGCG,24,DNA
3,1,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,None,NaN,...,38.45,None,NaN,1995.0,3,101D,B,CGCGAATTCGCG,24,DNA
4,2,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,None,NaN,...,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0,4,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,Protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471144,141399,9XIA,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,388,1.90,43542.29,None,NaN,...,55.93,None,NaN,1989.0,467299,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388,Protein
471145,141400,9XIM,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,1572,2.40,174722.12,None,NaN,...,68.92,None,NaN,1992.0,467300,9XIM,A,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572,Protein
471146,141400,9XIM,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,1572,2.40,174722.12,None,NaN,...,68.92,None,NaN,1992.0,467301,9XIM,B,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572,Protein
471147,141400,9XIM,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,1572,2.40,174722.12,None,NaN,...,68.92,None,NaN,1992.0,467302,9XIM,C,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572,Protein


ii. left join

In [17]:
query = '''
SELECT * FROM dups d
left join seq s
on d.structureId = s.structureId
'''
seq_data = query_data(query, engine)
seq_data

,index,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,...,densityPercentSol,pdbxDetails,phValue,publicationYear,index,structureId,chainId,sequence,residueCount,macromoleculeType
0,0,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,...,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0,0.0,100D,A,CCGGCGCCGG,20.0,DNA/RNA Hybrid
1,0,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,...,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0,1.0,100D,B,CCGGCGCCGG,20.0,DNA/RNA Hybrid
2,1,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,None,NaN,...,38.45,None,NaN,1995.0,2.0,101D,A,CGCGAATTCGCG,24.0,DNA
3,1,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,None,NaN,...,38.45,None,NaN,1995.0,3.0,101D,B,CGCGAATTCGCG,24.0,DNA
4,2,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,None,NaN,...,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0,4.0,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154.0,Protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471806,141399,9XIA,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,388,1.90,43542.29,None,NaN,...,55.93,None,NaN,1989.0,467299.0,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388.0,Protein
471807,141400,9XIM,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,1572,2.40,174722.12,None,NaN,...,68.92,None,NaN,1992.0,467300.0,9XIM,A,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572.0,Protein
471808,141400,9XIM,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,1572,2.40,174722.12,None,NaN,...,68.92,None,NaN,1992.0,467301.0,9XIM,B,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572.0,Protein
471809,141400,9XIM,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,1572,2.40,174722.12,None,NaN,...,68.92,None,NaN,1992.0,467302.0,9XIM,C,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572.0,Protein


#### EXERCISE #2

We see the small difference in the rows between two types of join. Create a table that contains these residual data. What can you infer from it?

In [18]:
#Answer goes here

### d. Aggregations and Conditionals
Dealing with data means you need to deal with statistics. SQL has a lot in its sleeve for the statistical tools. Let's list some of them here, where we take the molecular weight column (you can honestly google the rest):

In [19]:
query = '''
SELECT count(structureMolecularWeight) as count,
max(structureMolecularWeight) as max_weight,
min(structureMolecularWeight) as min_weight,
avg(structureMolecularWeight) as mean_weight,
sum(structureMolecularWeight) as sum_weight
FROM dups d
'''
seq_data = query_data(query, engine)
seq_data

,count,max_weight,min_weight,mean_weight,sum_weight
0,141401,97730536.0,314.38,112079.000614,1.584808e+10


The functions alone don't help that much. It is by using grouping functions that the aggregate functions shine. For example, let's group all of the above by its Structure ID:

In [20]:
query = '''
SELECT structureId,
count(structureMolecularWeight) as count,
max(structureMolecularWeight) as max_weight,
min(structureMolecularWeight) as min_weight,
avg(structureMolecularWeight) as mean_weight,
sum(structureMolecularWeight) as sum_weight
FROM dups d
group by 1
order by 1
'''
seq_data = query_data(query, engine)
seq_data

,structureId,count,max_weight,min_weight,mean_weight,sum_weight
0,100D,1,6360.30,6360.30,6360.30,6360.30
1,101D,1,7939.35,7939.35,7939.35,7939.35
2,101M,1,18112.80,18112.80,18112.80,18112.80
3,102D,1,7637.17,7637.17,7637.17,7637.17
4,102L,1,18926.61,18926.61,18926.61,18926.61
...,...,...,...,...,...,...
140906,9RUB,1,101838.68,101838.68,101838.68,101838.68
140907,9TNA,1,24244.34,24244.34,24244.34,24244.34
140908,9WGA,1,34270.22,34270.22,34270.22,34270.22
140909,9XIA,1,43542.29,43542.29,43542.29,43542.29


Another thing that makes aggregation shine is that it can be used in conditionals. It usually goes as:

CASE WHEN (statement) then (result) ELSE (another result) END

In [21]:
query = '''
SELECT structureId,
case when
max(structureMolecularWeight) < 8000
then 1 
else 0
end as is_big_molecule
FROM dups d
group by 1
order by 1
'''
seq_data = query_data(query, engine)
seq_data

,structureId,is_big_molecule
0,100D,1
1,101D,1
2,101M,0
3,102D,1
4,102L,0
...,...,...
140906,9RUB,0
140907,9TNA,0
140908,9WGA,0
140909,9XIA,0


### Exercise 3
You noticed there aren't any median aggregate function, right? Create one.

In [22]:
#Answer goes here

### Outlier Treatment

